In [1]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) 

import pandas as pd
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import os

import numpy as np
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

from sklearn.metrics import accuracy_score
from datetime import datetime
from IPython.display import clear_output 

Mounted at /content/drive


In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113
     |████████████████████████████████| 7.9 MB 3.8 MB/s 
     |████████████████████████████████| 3.5 MB 3.4 MB/s 
     |████████████████████████████████| 2.4 MB 4.5 MB/s 


In [3]:
train_dataset=[]
graphlist=[]
import networkx as nx
import torch_geometric
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
#from torch.utils.data import DataLoader
from torch_geometric.utils import degree
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx


%cd /content/drive/MyDrive/ADNI2/Controls
for i in range(1, 110):
    if os.path.isfile('{0:0>3}'.format(i)+'.txt'):
      tmp=pd.read_csv('{0:0>3}'.format(i)+'.txt', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp)  
      tmp[tmp<=0.4]=0
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      strength = G.degree(weight='weight')
      x = []
      for i in strength:
        x.extend([i[1]])
      x = np.array(x).reshape(90,1)
      data = from_networkx(G)
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = torch.FloatTensor(x)
      data.y = torch.tensor([1])
      graphlist.append(data)
      #graphlist.append([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])

%cd /content/drive/MyDrive/ADNI2/Patients
for i in range(1, 110):
    if os.path.isfile('{0:0>3}'.format(i)+'.txt'):
      tmp=pd.read_csv('{0:0>3}'.format(i)+'.txt', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp) 
      tmp[tmp<=0.4]=0
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      strength = G.degree(weight='weight')
      x = []
      for i in strength:
        x.extend([i[1]])
      x = np.array(x).reshape(90,1)
      data = from_networkx(G)
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = torch.FloatTensor(x)
      data.y = torch.tensor([0])
      graphlist.append(data)
      #graphlist.append([data, torch.zeros(1).type(torch.long)])

      #graphlist.append([tmp.type(torch.FloatTensor),torch.zeros(1).type(torch.long)])

/content/drive/.shortcut-targets-by-id/1n9P-S3rsi_g9yeb5KW1h7v-Bte7Cy43G/ADNI2/Controls
/content/drive/.shortcut-targets-by-id/1n9P-S3rsi_g9yeb5KW1h7v-Bte7Cy43G/ADNI2/Patients


In [4]:
graphlist[0]

Data(edge_index=[2, 3054], weight=[3054], num_nodes=90, x=[90, 1], y=[1])

In [5]:
%cd /content/drive/MyDrive
pos = pd.read_csv("node_aal90.csv", header = None)

/content/drive/MyDrive


In [6]:
position1 = []
for i in pos[0]:
  position1.extend([i])
position1 = np.array(position1).reshape(90,1)

position2 = []
for i in pos[1]:
  position2.extend([i])
position2 = np.array(position2).reshape(90,1)

position3 = []
for i in pos[2]:
  position3.extend([i])
position3 = np.array(position3).reshape(90,1)

position = np.concatenate((position1, position2, position3), axis = 1) 

In [7]:
for i in range(len(graphlist)):
  graphlist[i].x = torch.FloatTensor(np.concatenate((graphlist[i].x, position), axis = 1))

In [8]:
graphlist[0]

Data(edge_index=[2, 3054], weight=[3054], num_nodes=90, x=[90, 4], y=[1])

In [9]:
dataset = random.sample(graphlist, len(graphlist))
train_dataset = dataset[:150]
test_dataset = dataset[150:]
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=False)

In [10]:
from torch_geometric.nn import GATConv, GraphConv
from torch_geometric.nn.conv.gatv2_conv import GATv2Conv
from torch_geometric.nn import global_mean_pool, global_sort_pool

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()

        self.conv1 = GraphConv(in_channels=4, out_channels=16)
        self.conv2 = GraphConv(in_channels=16, out_channels=32)
        self.conv3 = GraphConv(in_channels=32, out_channels=32)
        self.lin1 = torch.nn.Linear(32, 16)
        self.lin2 = torch.nn.Linear(16, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch 
        x = self.conv1(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv2(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv3(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        #x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        #print(x)
        
        return x
        #return F.log_softmax(x, dim=1)

In [11]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4853, Loss: 0.6815
Epoch: 002, Train Acc: 0.5000, Test Acc: 0.4559, Loss: 0.9729
Epoch: 003, Train Acc: 0.6600, Test Acc: 0.7353, Loss: 0.6687
Epoch: 004, Train Acc: 0.6400, Test Acc: 0.7353, Loss: 0.5611
Epoch: 005, Train Acc: 0.6000, Test Acc: 0.6618, Loss: 0.5214
Epoch: 006, Train Acc: 0.7467, Test Acc: 0.8088, Loss: 0.4968
Epoch: 007, Train Acc: 0.7667, Test Acc: 0.8235, Loss: 0.4012
Epoch: 008, Train Acc: 0.7467, Test Acc: 0.8382, Loss: 0.4133
Epoch: 009, Train Acc: 0.8133, Test Acc: 0.8529, Loss: 0.4572
Epoch: 010, Train Acc: 0.8067, Test Acc: 0.8676, Loss: 0.4540
Epoch: 011, Train Acc: 0.7933, Test Acc: 0.8529, Loss: 0.3253
Epoch: 012, Train Acc: 0.8267, Test Acc: 0.8529, Loss: 0.3247
Epoch: 013, Train Acc: 0.8333, Test Acc: 0.8529, Loss: 0.2568
Epoch: 014, Train Acc: 0.8333, Test Acc: 0.8382, Loss: 0.3614
Epoch: 015, Train Acc: 0.8467, Test Acc: 0.8382, Loss: 0.5240
Epoch: 016, Train Acc: 0.8267, Test Acc: 0.8382, Loss: 0.3295
Epoch: 0

In [12]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GCNConv, JumpingKnowledge, global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.num_layers  = 3
        self.hidden = 16
        self.conv1 = GCNConv(4, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(GCNConv(self.hidden, self.hidden))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index,weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index, weight))
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [13]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 500):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.7133, Test Acc: 0.7500, Loss: 0.6826
Epoch: 002, Train Acc: 0.5467, Test Acc: 0.5294, Loss: 0.6815
Epoch: 003, Train Acc: 0.8800, Test Acc: 0.8088, Loss: 0.5296
Epoch: 004, Train Acc: 0.8867, Test Acc: 0.8088, Loss: 0.4920
Epoch: 005, Train Acc: 0.9067, Test Acc: 0.8971, Loss: 0.3819
Epoch: 006, Train Acc: 0.9200, Test Acc: 0.9118, Loss: 0.2523
Epoch: 007, Train Acc: 0.8933, Test Acc: 0.9265, Loss: 0.2210
Epoch: 008, Train Acc: 0.9267, Test Acc: 0.9118, Loss: 0.2302
Epoch: 009, Train Acc: 0.9067, Test Acc: 0.9265, Loss: 0.1926
Epoch: 010, Train Acc: 0.9267, Test Acc: 0.9265, Loss: 0.3900
Epoch: 011, Train Acc: 0.9267, Test Acc: 0.9412, Loss: 0.1902
Epoch: 012, Train Acc: 0.9200, Test Acc: 0.9265, Loss: 0.1783
Epoch: 013, Train Acc: 0.9267, Test Acc: 0.9265, Loss: 0.4499
Epoch: 014, Train Acc: 0.9400, Test Acc: 0.9412, Loss: 0.0703
Epoch: 015, Train Acc: 0.9333, Test Acc: 0.9265, Loss: 0.0713
Epoch: 016, Train Acc: 0.9400, Test Acc: 0.9412, Loss: 0.0683
Epoch: 0

In [14]:
from torch.nn import Linear
from torch_geometric.nn import GINConv, JumpingKnowledge, global_mean_pool
from torch.nn import BatchNorm1d as BN
from torch.nn import Linear, ReLU, Sequential
#from torch_geometric.nn import JumpingKnowledge, SAGEConv, global_mean_pool

class GIN(torch.nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(
                Linear(4, 16),
                ReLU(),
                Linear(16, 16),
                ReLU(),
                BN(16),
            ), train_eps=False)
        self.conv2 = GINConv(
            Sequential(
                Linear(16, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.conv3 = GINConv(
            Sequential(
                Linear(32, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.lin1 = Linear(32, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [15]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GIN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 200):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5600, Test Acc: 0.6618, Loss: 0.4643
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.4853, Loss: 0.4362
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.4853, Loss: 0.1910
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.4853, Loss: 0.2265
Epoch: 005, Train Acc: 0.9467, Test Acc: 0.9118, Loss: 0.1394
Epoch: 006, Train Acc: 0.8800, Test Acc: 0.7941, Loss: 0.0893
Epoch: 007, Train Acc: 0.7667, Test Acc: 0.7206, Loss: 0.4952
Epoch: 008, Train Acc: 0.7867, Test Acc: 0.8529, Loss: 0.4175
Epoch: 009, Train Acc: 0.8400, Test Acc: 0.8235, Loss: 0.0979
Epoch: 010, Train Acc: 0.7400, Test Acc: 0.6765, Loss: 0.1684
Epoch: 011, Train Acc: 0.9600, Test Acc: 0.8971, Loss: 0.1573
Epoch: 012, Train Acc: 0.9533, Test Acc: 0.8824, Loss: 0.1433
Epoch: 013, Train Acc: 0.7400, Test Acc: 0.6176, Loss: 0.2218
Epoch: 014, Train Acc: 0.8800, Test Acc: 0.9118, Loss: 0.1637
Epoch: 015, Train Acc: 0.9400, Test Acc: 0.8529, Loss: 0.5533
Epoch: 016, Train Acc: 0.7267, Test Acc: 0.5294, Loss: 0.0509
Epoch: 0

In [16]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.data import Batch
from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
    graclus,
    max_pool,
)

num_layers  = 4
hidden = 16
class Graclus(torch.nn.Module):
    def __init__(self):
        super(Graclus, self).__init__()
        self.conv1 = GraphConv(4, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(GraphConv(16, 16, aggr='mean'))
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(64, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, weight , batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                cluster = graclus(edge_index,num_nodes=x.size(0))
                data = Batch(x=x, edge_index=edge_index, batch=batch)
                data = max_pool(cluster, data)
                x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

In [17]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Graclus().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 500):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4933, Test Acc: 0.4853, Loss: 0.6845
Epoch: 002, Train Acc: 0.6067, Test Acc: 0.6618, Loss: 0.6230
Epoch: 003, Train Acc: 0.7733, Test Acc: 0.8824, Loss: 0.6453
Epoch: 004, Train Acc: 0.8400, Test Acc: 0.8676, Loss: 0.6164
Epoch: 005, Train Acc: 0.6800, Test Acc: 0.8382, Loss: 0.6660
Epoch: 006, Train Acc: 0.6667, Test Acc: 0.6324, Loss: 0.6079
Epoch: 007, Train Acc: 0.7133, Test Acc: 0.8235, Loss: 0.5606
Epoch: 008, Train Acc: 0.8400, Test Acc: 0.8529, Loss: 0.6023
Epoch: 009, Train Acc: 0.8133, Test Acc: 0.8676, Loss: 0.4796
Epoch: 010, Train Acc: 0.7733, Test Acc: 0.8676, Loss: 0.5394
Epoch: 011, Train Acc: 0.8533, Test Acc: 0.9118, Loss: 0.5558
Epoch: 012, Train Acc: 0.8333, Test Acc: 0.8971, Loss: 0.4168
Epoch: 013, Train Acc: 0.9067, Test Acc: 0.8382, Loss: 0.4259
Epoch: 014, Train Acc: 0.8933, Test Acc: 0.8824, Loss: 0.5119
Epoch: 015, Train Acc: 0.7267, Test Acc: 0.8824, Loss: 0.4903
Epoch: 016, Train Acc: 0.8600, Test Acc: 0.7206, Loss: 0.4146
Epoch: 0

In [18]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    TopKPooling,
    global_mean_pool,
)


class TopK(torch.nn.Module):
    def __init__(self):
        super(TopK, self).__init__()
        self.ratio = 0.8
        self.num_layers = 3
        self.hidden = 16
        self.conv1 = GraphConv(4, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(16, 16, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [TopKPooling(16, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(48, 16)
        self.lin2 = Linear(16, 1)


    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        #return F.log_softmax(x, dim=-1)
        return x

In [19]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TopK().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4853, Loss: 0.7338
Epoch: 002, Train Acc: 0.7667, Test Acc: 0.8382, Loss: 0.6444
Epoch: 003, Train Acc: 0.6933, Test Acc: 0.7794, Loss: 0.6485
Epoch: 004, Train Acc: 0.8133, Test Acc: 0.7500, Loss: 0.6370
Epoch: 005, Train Acc: 0.8733, Test Acc: 0.8676, Loss: 0.5937
Epoch: 006, Train Acc: 0.7867, Test Acc: 0.8676, Loss: 0.5762
Epoch: 007, Train Acc: 0.8333, Test Acc: 0.8824, Loss: 0.5697
Epoch: 008, Train Acc: 0.8467, Test Acc: 0.9118, Loss: 0.6105
Epoch: 009, Train Acc: 0.8467, Test Acc: 0.9265, Loss: 0.5990
Epoch: 010, Train Acc: 0.8533, Test Acc: 0.8824, Loss: 0.5121
Epoch: 011, Train Acc: 0.8333, Test Acc: 0.8824, Loss: 0.5037
Epoch: 012, Train Acc: 0.8867, Test Acc: 0.9265, Loss: 0.4553
Epoch: 013, Train Acc: 0.8533, Test Acc: 0.9118, Loss: 0.4240
Epoch: 014, Train Acc: 0.9067, Test Acc: 0.9412, Loss: 0.3828
Epoch: 015, Train Acc: 0.8600, Test Acc: 0.8971, Loss: 0.4102
Epoch: 016, Train Acc: 0.8933, Test Acc: 0.9559, Loss: 0.3343
Epoch: 0

In [20]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    SAGPooling,
    global_mean_pool,
)


class SAGPool(torch.nn.Module):
    def __init__(self):
        super(SAGPool, self).__init__()
        self.num_layers = 4
        self.hidden = 32
        self.ratio = 0.8
        self.conv1 = GraphConv(4, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [SAGPooling(self.hidden, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [21]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4853, Loss: 0.7097
Epoch: 002, Train Acc: 0.9400, Test Acc: 0.8971, Loss: 0.6460
Epoch: 003, Train Acc: 0.9333, Test Acc: 0.8971, Loss: 0.6415
Epoch: 004, Train Acc: 0.9267, Test Acc: 0.8824, Loss: 0.5604
Epoch: 005, Train Acc: 0.9267, Test Acc: 0.8971, Loss: 0.5821
Epoch: 006, Train Acc: 0.9067, Test Acc: 0.8529, Loss: 0.4185
Epoch: 007, Train Acc: 0.8067, Test Acc: 0.8382, Loss: 0.3748
Epoch: 008, Train Acc: 0.8533, Test Acc: 0.8971, Loss: 0.4978
Epoch: 009, Train Acc: 0.9267, Test Acc: 0.9118, Loss: 0.3994
Epoch: 010, Train Acc: 0.8467, Test Acc: 0.8971, Loss: 0.3101
Epoch: 011, Train Acc: 0.9267, Test Acc: 0.9118, Loss: 0.4940
Epoch: 012, Train Acc: 0.9267, Test Acc: 0.9118, Loss: 0.5117
Epoch: 013, Train Acc: 0.9133, Test Acc: 0.9118, Loss: 0.2759
Epoch: 014, Train Acc: 0.9267, Test Acc: 0.8971, Loss: 0.2333
Epoch: 015, Train Acc: 0.9133, Test Acc: 0.9118, Loss: 0.2147
Epoch: 016, Train Acc: 0.9200, Test Acc: 0.9118, Loss: 0.4387
Epoch: 0

In [22]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    EdgePooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class EdgePool(torch.nn.Module):
    def __init__(self):
        super(EdgePool, self).__init__()
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = GraphConv(4, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [EdgePooling(self.hidden) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight ,  batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, batch, _ = pool(x, edge_index, batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [23]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EdgePool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4933, Test Acc: 0.5147, Loss: 0.6422
Epoch: 002, Train Acc: 0.8733, Test Acc: 0.8529, Loss: 0.6298
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.4853, Loss: 0.6592
Epoch: 004, Train Acc: 0.8600, Test Acc: 0.8529, Loss: 0.6275
Epoch: 005, Train Acc: 0.9000, Test Acc: 0.8235, Loss: 0.5882
Epoch: 006, Train Acc: 0.8867, Test Acc: 0.7647, Loss: 0.4951
Epoch: 007, Train Acc: 0.9267, Test Acc: 0.9118, Loss: 0.4542
Epoch: 008, Train Acc: 0.8600, Test Acc: 0.8676, Loss: 0.2938
Epoch: 009, Train Acc: 0.9400, Test Acc: 0.9412, Loss: 0.3047
Epoch: 010, Train Acc: 0.9467, Test Acc: 0.9559, Loss: 0.4278
Epoch: 011, Train Acc: 0.9267, Test Acc: 0.8529, Loss: 0.1552
Epoch: 012, Train Acc: 0.9267, Test Acc: 0.9412, Loss: 0.3030
Epoch: 013, Train Acc: 0.9400, Test Acc: 0.9412, Loss: 0.4755
Epoch: 014, Train Acc: 0.9400, Test Acc: 0.9412, Loss: 0.2410
Epoch: 015, Train Acc: 0.9333, Test Acc: 0.9265, Loss: 0.2352
Epoch: 016, Train Acc: 0.9400, Test Acc: 0.9118, Loss: 0.3479
Epoch: 0

In [24]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GlobalAttention, SAGEConv


class GlobalAttentionNet(torch.nn.Module):
    def __init__(self):
        super(GlobalAttentionNet, self).__init__()
        self.num_layers = 3
        self.hidden  = 16
        self.conv1 = SAGEConv(4, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.att = GlobalAttention(Linear(self.hidden, 1))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.att(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [25]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GlobalAttentionNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4933, Test Acc: 0.5147, Loss: 0.8095


/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


Epoch: 002, Train Acc: 0.4933, Test Acc: 0.5147, Loss: 0.7881
Epoch: 003, Train Acc: 0.4933, Test Acc: 0.5147, Loss: 0.7733
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.5147, Loss: 0.7061
Epoch: 005, Train Acc: 0.4933, Test Acc: 0.5147, Loss: 0.7731
Epoch: 006, Train Acc: 0.4933, Test Acc: 0.5147, Loss: 0.7146
Epoch: 007, Train Acc: 0.4933, Test Acc: 0.5147, Loss: 0.7509
Epoch: 008, Train Acc: 0.4933, Test Acc: 0.5147, Loss: 0.7732
Epoch: 009, Train Acc: 0.4933, Test Acc: 0.5147, Loss: 0.6464
Epoch: 010, Train Acc: 0.4933, Test Acc: 0.5147, Loss: 0.6602
Epoch: 011, Train Acc: 0.4933, Test Acc: 0.5147, Loss: 0.6982
Epoch: 012, Train Acc: 0.4933, Test Acc: 0.5147, Loss: 0.7408
Epoch: 013, Train Acc: 0.5000, Test Acc: 0.5147, Loss: 0.6704
Epoch: 014, Train Acc: 0.5000, Test Acc: 0.5294, Loss: 0.6713
Epoch: 015, Train Acc: 0.5333, Test Acc: 0.5441, Loss: 0.6807
Epoch: 016, Train Acc: 0.5933, Test Acc: 0.5735, Loss: 0.6577
Epoch: 017, Train Acc: 0.6200, Test Acc: 0.6471, Loss: 0.6628
Epoch: 0

In [26]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import SAGEConv, Set2Set


class Set2SetNet(torch.nn.Module):
    def __init__(self):
        super(Set2SetNet,self).__init__()
        self.hidden = 16
        self.num_layers = 3 
        self.conv1 = SAGEConv(4, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.set2set = Set2Set(self.hidden, processing_steps=4)
        self.lin1 = Linear(2 * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.set2set(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [27]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Set2SetNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.Set2Set' is deprecated, use 'nn.aggr.Set2Set' instead
  warnings.warn(out)


Epoch: 001, Train Acc: 0.5533, Test Acc: 0.5147, Loss: 0.6830
Epoch: 002, Train Acc: 0.7800, Test Acc: 0.7500, Loss: 0.6445
Epoch: 003, Train Acc: 0.8800, Test Acc: 0.8676, Loss: 0.6118
Epoch: 004, Train Acc: 0.9067, Test Acc: 0.8676, Loss: 0.5929
Epoch: 005, Train Acc: 0.9133, Test Acc: 0.9412, Loss: 0.4670
Epoch: 006, Train Acc: 0.9200, Test Acc: 0.8971, Loss: 0.3563
Epoch: 007, Train Acc: 0.9200, Test Acc: 0.8824, Loss: 0.3327
Epoch: 008, Train Acc: 0.9400, Test Acc: 0.9118, Loss: 0.2205
Epoch: 009, Train Acc: 0.9267, Test Acc: 0.9412, Loss: 0.4231
Epoch: 010, Train Acc: 0.9067, Test Acc: 0.8971, Loss: 0.2122
Epoch: 011, Train Acc: 0.9133, Test Acc: 0.9265, Loss: 0.3292
Epoch: 012, Train Acc: 0.9133, Test Acc: 0.8971, Loss: 0.3211
Epoch: 013, Train Acc: 0.9267, Test Acc: 0.9118, Loss: 0.3586
Epoch: 014, Train Acc: 0.9400, Test Acc: 0.9118, Loss: 0.1137
Epoch: 015, Train Acc: 0.9400, Test Acc: 0.9265, Loss: 0.2814
Epoch: 016, Train Acc: 0.9400, Test Acc: 0.9265, Loss: 0.1002
Epoch: 0

In [28]:
import torch
import torch.nn.functional as F
from torch.nn import Conv1d, Linear

from torch_geometric.nn import SAGEConv, global_sort_pool


class SortPool(torch.nn.Module):
    def __init__(self):
        super(SortPool, self).__init__()
        self.k = 30
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = SAGEConv(4, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.conv1d = Conv1d(self.hidden, 32, 5)
        self.lin1 = Linear(32 * (self.k - 5 + 1), self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = global_sort_pool(x, batch, self.k)
        x = x.view(len(x), self.k, -1).permute(0, 2, 1)
        x = F.relu(self.conv1d(x))
        x = x.view(len(x), -1)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [29]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SortPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.global_sort_pool' is deprecated, use 'nn.aggr.SortAggr' instead
  warnings.warn(out)


Epoch: 001, Train Acc: 0.5867, Test Acc: 0.5882, Loss: 0.6506
Epoch: 002, Train Acc: 0.6600, Test Acc: 0.6324, Loss: 0.6638
Epoch: 003, Train Acc: 0.6067, Test Acc: 0.5882, Loss: 0.6680
Epoch: 004, Train Acc: 0.7467, Test Acc: 0.7059, Loss: 0.6360
Epoch: 005, Train Acc: 0.7733, Test Acc: 0.8529, Loss: 0.6523
Epoch: 006, Train Acc: 0.7133, Test Acc: 0.7059, Loss: 0.6296
Epoch: 007, Train Acc: 0.8333, Test Acc: 0.8382, Loss: 0.6320
Epoch: 008, Train Acc: 0.8800, Test Acc: 0.8088, Loss: 0.5977
Epoch: 009, Train Acc: 0.8933, Test Acc: 0.8529, Loss: 0.5609
Epoch: 010, Train Acc: 0.8933, Test Acc: 0.8676, Loss: 0.5628
Epoch: 011, Train Acc: 0.8933, Test Acc: 0.8824, Loss: 0.5400
Epoch: 012, Train Acc: 0.8933, Test Acc: 0.8824, Loss: 0.4980
Epoch: 013, Train Acc: 0.9067, Test Acc: 0.8529, Loss: 0.4684
Epoch: 014, Train Acc: 0.9200, Test Acc: 0.8676, Loss: 0.5340
Epoch: 015, Train Acc: 0.8933, Test Acc: 0.8382, Loss: 0.4119
Epoch: 016, Train Acc: 0.9333, Test Acc: 0.8676, Loss: 0.4645
Epoch: 0

In [30]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    ASAPooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class ASAP(torch.nn.Module):
    def __init__(self):
        super(ASAP, self).__init__()
        self.num_layers = 5 
        self.hidden  = 32
        self.ratio = 0.8
        self.dropout = 0.2
        self.conv1 = GraphConv(4, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend([
            ASAPooling(self.hidden, self.ratio, dropout=self.dropout)
            for i in range((self.num_layers) // 2)
        ])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index,data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = conv(x=x, edge_index=edge_index, edge_weight=edge_weight)
            x = F.relu(x)
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, edge_weight, batch, _ = pool(
                    x=x, edge_index=edge_index, edge_weight=edge_weight,
                    batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [31]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ASAP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4853, Loss: 0.6815
Epoch: 002, Train Acc: 0.8067, Test Acc: 0.8235, Loss: 0.6786
Epoch: 003, Train Acc: 0.5800, Test Acc: 0.6176, Loss: 0.6750
Epoch: 004, Train Acc: 0.6600, Test Acc: 0.7794, Loss: 0.6880
Epoch: 005, Train Acc: 0.8600, Test Acc: 0.8235, Loss: 0.6677
Epoch: 006, Train Acc: 0.8533, Test Acc: 0.7647, Loss: 0.6729
Epoch: 007, Train Acc: 0.8533, Test Acc: 0.8088, Loss: 0.6541
Epoch: 008, Train Acc: 0.8867, Test Acc: 0.8235, Loss: 0.6593
Epoch: 009, Train Acc: 0.8067, Test Acc: 0.8824, Loss: 0.6438
Epoch: 010, Train Acc: 0.8267, Test Acc: 0.8676, Loss: 0.6329
Epoch: 011, Train Acc: 0.8267, Test Acc: 0.8824, Loss: 0.6577
Epoch: 012, Train Acc: 0.8800, Test Acc: 0.8676, Loss: 0.6505
Epoch: 013, Train Acc: 0.8533, Test Acc: 0.8824, Loss: 0.6236
Epoch: 014, Train Acc: 0.8267, Test Acc: 0.8824, Loss: 0.6520
Epoch: 015, Train Acc: 0.8267, Test Acc: 0.8824, Loss: 0.6434
Epoch: 016, Train Acc: 0.8267, Test Acc: 0.8971, Loss: 0.6203
Epoch: 0